In [ ]:
import os
import json
import random
from glob import glob
from typing import Mapping, Literal, Any

import textworld.gym
from textworld import EnvInfos, Agent

import numpy as np
import pandas as pd
import ollama

from IPython.display import clear_output

from langgraph.graph import MessagesState
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage

from langgraph.graph import START, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display

from langchain_core.tools import tool

from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

from langgraph.checkpoint.memory import MemorySaver

from langgraph.graph import END

In [2]:
np.random.seed(42)
random.seed(42)
SEEDS = [random.randint(0, 1000) for _ in range(5)]
print(SEEDS)
BPMNS = sorted([x for x in os.listdir("bpmn") if x.endswith(".bpmn")])
print(BPMNS)
BPMN_PATHS = [os.path.join("bpmn", x) for x in BPMNS]
print(BPMN_PATHS)

if not os.path.isdir("descriptions"):
    os.makedirs("descriptions")

[654, 114, 25, 759, 281]
['process1.bpmn', 'process2.bpmn', 'process3.bpmn', 'process4.bpmn', 'process5.bpmn', 'process6.bpmn', 'process7.bpmn']
['bpmn/process1.bpmn', 'bpmn/process2.bpmn', 'bpmn/process3.bpmn', 'bpmn/process4.bpmn', 'bpmn/process5.bpmn', 'bpmn/process6.bpmn', 'bpmn/process7.bpmn']


In [3]:
for bpmn_path in BPMN_PATHS:
    for seed in SEEDS:    
        llm = ChatOpenAI(
            model="gpt-4o",
            temperature=0,
            seed=seed
        )
        process_bpmn = ""
        with open(bpmn_path, "r") as f:
            process_bpmn = f.read()
        process_rule = llm.invoke(f"""Convert the following BPMN XML into a natural, human-readable text that describes the process as if someone were explaining it in plain language. The text should be coherent and continuous, reflecting how a person would describe the process step by step.  

- Only include tasks performed by the LLM Agent. Ignore all other lanes.  
- Do not format the steps as a numbered list or in a rigid structure—write the explanation in flowing prose.  
- Avoid overly technical jargon. The description should be understandable to someone without BPMN knowledge.  
- Do not start with receiving the error message; begin the description from the next relevant action.  
- Ensure that all additional text annotations (e.g., error messages, emails, transaction details) are naturally incorporated into the explanation, maintaining the context in which they are used.
- Always include the EXPLICIT email-adresses of the persons to send the email to.
- Always include the EXPLICIT content to send in an email included in the text annotations.
- Always include the EXPLICIT transaction details of the transaction to be performed.
- Always include the EXPLICIT parameter names to be passed to the function in the text annotations.
- Always include the EXPLICIT parameter values to be passed to the function in the text annotations.
- Always include the EXPLICIT function to be called in the text annotations.

Use the following BPMN XML to generate the text:

{process_bpmn}

""").content
        rule_path = os.path.join("descriptions", f"{bpmn_path.split('/')[-1].replace('.bpmn', '')}")
        if not os.path.isdir(rule_path):
            os.makedirs(rule_path)
        with open(os.path.join(rule_path, f"{seed}.md"), "w") as f:
            f.write(process_rule)